In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords

data=pd.read_excel('Input_Path') 

stopwords = stopwords.words('english')

corpus=[]

for index,row in data.iterrows():
    corpus.append(row['Comments']) 
    

data['Comments'] = data['Comments'].astype("str")
data['Comments'] = data['Comments'].str.lower()
data['Comments'] = data['Comments'].str.replace(".","")
data['Comments'] = data['Comments'].str.replace(",","")
data['Comments'] = data['Comments'].str.replace("?","")
data['Comments'] = data['Comments'].str.replace("/","")
data['Comments'] = data['Comments'].str.replace("-","")
data['Comments'] = data['Comments'].str.replace(":","")
data['Comments'] = data['Comments'].str.replace("!","")
data['Comments'] = data['Comments'].str.replace("&","")
data['Comments'] = data['Comments'].str.replace("(","")
data['Comments'] = data['Comments'].str.replace(")","")


data['Comments'] = data['Comments'].apply(nltk.word_tokenize)

for i in range(0,134):
    for word in data['Comments'][i]:
        if word in stopwords:
            data['Comments'][i].remove(word)

            
data['Comments'] = data['Comments'].apply(', '.join)
data['Comments'] = data['Comments'].str.replace(",","")

idea=data['Comments'] #Selecting the first column that has text. 

# corpus=[]
# for i in data["Resolution"]:
#     corpus.append(i) 
#Converting the column of data from excel sheet into a list of documents, where each document corresponds to a group of sentences.
#Count Vectoriser then tidf transformer

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(idea.values.astype('U')) #ERROR AFTER EXECUTING THESE #LINES

#vectorizer.get_feature_names()
#print(X.toarray())     

from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(X)
print(tfidf.shape )                        

from sklearn.cluster import KMeans
num_clusters = 5 #Change it according to your data.
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf)
clusters = km.labels_.tolist()

idea={'Idea':corpus, 'Cluster':clusters} #Creating dict having doc with the corresponding cluster number.
frame=pd.DataFrame(idea,index=[clusters], columns=['Idea','Cluster']) # Converting it into a dataframe.

print("\n")
print(frame) #Print the doc with the labeled cluster number.
print("\n")

frame.to_excel("Output_Path")